# import stuff

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
#import seaborn as sns
#from matplotlib import pyplot as plt
#%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import rpy2
#import pingouin as pg
from itertools import combinations
import openpyxl

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.rinterface_lib.embedded import RRuntimeError

In [2]:
#lavaan = importr('lavaan')

In [3]:
# fix from here https://github.com/rpy2/rpy2/issues/882
import os
os.environ['R_HOME'] = '/Users/zeleninam2/miniconda3/envs/env_hitop_MYENVcfa/bin/R' # change this to whatever is relevant to you, or you might not need this line at all.
rbase = importr('base')
utils = importr('utils')

In [4]:
utils = importr('utils')
#quadprog = importr('quadprog')
lavaan = importr('lavaan')
semtools = importr('semTools')
stringr = importr('stringr')
reticulate = importr('reticulate')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [5]:
# importing stuff the right way
try:
    semTools_version = '.'.join([str(ii) for ii in list(ro.r('packageVersion("semTools")')[0])])
except RRuntimeError:
    semTools_version = ''
# There's a bug in the main semtools libarary, this install from my fork of semtools where I've fixed it.
if semTools_version != '0.5.6.933':
    ro.r("""
        library(devtools)
        install_github("shotgunosine/semTools/semTools@aa68e38f88d9dd881617d5ade429bde2b2a74e08")
        """
    )

# Paths

In [ ]:
path_to_item_lookup = '../../data/ValSample/Internalizing-Somatoform Items_DW.xlsx'
path_to_cogmood_questions = '../../data/CFA/cogmood_questions.csv'
path_to_valsample = '../../data/ValSample/inters23.xlsx'
path_to_codesheet = '../../dylan_github/yougov_codesheet_alignment.tsv'
path_to_genpop = '../../data/NIMH0007_genpop_num_OUTPUT.csv'
path_to_enriched = '../../data/NIMH0007_mental_health_num_OUTPUT.csv'

# Functions

In [6]:
def make_hitop_list(data):
    # only hitop cols
    hitop_cols = []
    for c in data.columns:
        if 'hitop' in c and 'today' not in c:
            hitop_cols.append(c)   
    return(hitop_cols)

def make_recontact_list(data):
    list_recontact = []
    for c in data.columns:
        if 'recontact' in c:
            list_recontact.append(c)
    return(list_recontact)

def do_checks(dat, remove_checks):
    assert remove_checks in ['no', 'yes', 'grid', 'gridinitialvisitonly']
    dat['passed_checks'] = True
    dat['passed_grid'] = True
    dat['passed_grid_initialonly'] = True
    dat['passed_list'] = True
    dat.loc[dat.check_moderately != 3, 'passed_checks'] = False
    dat.loc[dat.check_notatall != 1, 'passed_checks'] = False
    dat.loc[dat.check_moderately != 3, 'passed_grid'] = False
    dat.loc[dat.check_notatall != 1, 'passed_grid'] = False
    dat.loc[dat.todaycheck_1 != 1, 'passed_checks'] = False
    dat.loc[dat.todaycheck_2 != 1, 'passed_checks'] = False
    dat.loc[dat.todaycheck_1 != 1, 'passed_list'] = False
    dat.loc[dat.todaycheck_2 != 1, 'passed_list'] = False
    # deal with attention checks - recontact
    dat['passed_checks_recontact'] = True
    dat['passed_grid_recontact'] = True
    dat['passed_list_recontact'] = True
    dat.loc[dat.check_moderately_recontact != 3, 'passed_checks_recontact'] = False
    dat.loc[dat.check_notatall_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.check_moderately_recontact != 3, 'passed_grid_recontact'] = False
    dat.loc[dat.check_notatall_recontact != 1, 'passed_grid_recontact'] = False
    dat.loc[dat.todaycheck_1_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.todaycheck_2_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.todaycheck_1_recontact != 1, 'passed_list_recontact'] = False
    dat.loc[dat.todaycheck_2_recontact != 1, 'passed_list_recontact'] = False
    print('checks_initial:')
    print(1 - dat.loc[:, ['passed_checks', 'passed_grid', 'passed_list']].mean())
    how_many_passed_checks = dat['passed_checks'].value_counts()[True]
    how_many_overall = dat.shape[0]
    print('Passed checks: %d out of %d' % (how_many_passed_checks, how_many_overall))
    print('checks_recontact:')
    print(1 - dat.loc[:, ['passed_checks_recontact', 'passed_grid_recontact', 'passed_list_recontact']].mean()) 
    how_many_passed_checks_recontact = dat['passed_checks_recontact'].value_counts()[True]
    print('Passed checks: %d out of %d' % (how_many_passed_checks_recontact, how_many_overall))
    if remove_checks == 'yes':
        print('removing checks')
        dat = dat.loc[dat['passed_checks'] == True]
        dat = dat.loc[dat['passed_checks_recontact'] == True]
        print(dat.shape)
        print('done removing checks')
    elif remove_checks == 'grid':
        print('removing checks')
        dat = dat.loc[dat['passed_grid'] == True]
        dat = dat.loc[dat['passed_grid_recontact'] == True]
        print(dat.shape)
        print('done removing checks') 
    elif remove_checks == 'gridinitialvisitonly':
        print('removing checks')
        dat = dat.loc[dat['passed_grid'] == True]
        print(dat.shape)
        print('done removing checks')        
    return(dat)

def load_item_lookup():
    # this is a table that aligns scale items with their corresponding questions
    htqs = pd.read_excel(path_to_item_lookup, skiprows=2,
                                names=['id', 'scale', 'item', '_0', '_1', '_2', '_3']).drop(['_0', '_1', '_2', '_3'], axis=1)
    # cmqs is a dataframe with items and responses
    # htcm is the same dataframe, but only displaying HiTOP items
    cmqs = pd.read_csv(path_to_cogmood_questions)
    cmqs = cmqs.rename({'Unnamed: 0': 'id'}, axis=1)
    htcm = cmqs.query("measure == 'HiTOP'")
    # this is a lookup table for subscales, items (sentence questions), and hitop ids
    item_lookup = htcm.loc[:, ['id','subscale', 'item']].merge(htqs.loc[:, ['id','item']], how='inner', on='item', suffixes=['_cm', '_ht'])
    item_lookup['htid'] = 'hitop'+ item_lookup.id_ht.astype(str)
    return(item_lookup)
    
def load_valsample():
    # VALIDATION SAMPLE - main dataframe with responses
    dat_validation = pd.read_excel(path_to_valsample)
    '''resp_cols = responses.columns[responses.columns.isin(item_lookup.htid)]
    # scale scores
    scale_scores = []
    for ss, df in item_lookup.groupby('subscale'):
        tmp = responses.loc[:, df.htid].sum(1)
        tmp.name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        scale_scores.append(tmp)
    scale_scores = pd.concat(scale_scores, axis=1)
    scale_names = list(scale_scores.columns.values)'''
    #do_checks(dat_validation)    
    # this is the validation data to be used
    hitop_cols_list = make_hitop_list(dat_validation)
    data_validation = dat_validation[hitop_cols_list]
    data_validation = data_validation.assign(whichdata=['validation']*data_validation.shape[0])
    return(data_validation)

def do_codesheet():
    #loads the codesheet file
    # wherever the emane column exists, creates a dict variablename:ename
    code_book = pd.read_csv(path_to_codesheet, sep='\t')
    ename_lut = {vn:en for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values}
    for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values:
        ename_lut[vn+'_recontact'] = en+'_recontact'
    tmp = code_book.query('ename.notnull()')
    scale_lut = {}
    for ss, df in tmp.loc[tmp.ename.str.contains('hitop') & ~tmp.ename.str.contains('today')].groupby('subscale'):
        scale_name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = df.ename.values
        scale_lut[scale_name] = items    
     #   scale_name_initial = scale_name+'_initial'
     #   scale_lut[scale_name_initial] = items 
        scale_name_recontact = scale_name+'_recontact'
        items_recontact = []
        for item in items:
            items_recontact.append(item+'_recontact')
        scale_lut[scale_name_recontact] = np.array(items_recontact, dtype=object)
    # rename checks
    ename_lut['FNM_Q8_5'] ='check_moderately'
    ename_lut['FNM_Q22_3'] ='check_notatall'
    ename_lut['FNM_Q42_m_10'] ='todaycheck_1'
    ename_lut['FNM_44_m_28'] ='todaycheck_2'
    ename_lut['FNM_Q8_5_recontact'] ='check_moderately_recontact'
    ename_lut['FNM_Q22_3_recontact'] ='check_notatall_recontact'
    ename_lut['FNM_Q42_m_10_recontact'] ='todaycheck_1_recontact'
    ename_lut['FNM_44_m_28_recontact'] ='todaycheck_2_recontact'
    return ename_lut, scale_lut
    
def load_data(dataname, doing_checks, doing_remove_checks):
    print(doing_remove_checks)
    assert(dataname in ['genpop','enriched'])
    #codesheet file
    ename_lut, scale_lut = do_codesheet()
    # done working with the codesheet file, now opening the actual data
    if dataname == 'genpop':
        datapath = path_to_genpop
    elif dataname == 'enriched':
        datapath = path_to_enriched
    dat = pd.read_csv(datapath, dtype={'caseid':str}, engine='python')
    #print(dat.head())
    if dataname == 'genpop':
        # ONLY GENPOP drop the .0 that pandas appends for some reason - only for genpop
        dat['caseid'] = dat.caseid.str[:-2]
    dat = dat.rename(ename_lut, axis=1)
    if doing_checks:
        dat = do_checks(dat = dat, remove_checks = doing_remove_checks) 
    hitop_cols_list = make_hitop_list(dat)
    data = dat[hitop_cols_list]
    data = data.assign(whichdata=[dataname]*data.shape[0])
    recontact_cols_list = make_recontact_list(data)
    data_norecontact = data.drop(columns=recontact_cols_list)
    return data, data_norecontact   

def prep_data_recontactanalysis(dataframe_withrecontact):
    df = dataframe_withrecontact
    df_helper = df.copy(deep=True)
    df_rows = df.shape[0]
    df = df.assign(whichvisit=['initialvisit']*df_rows)
    df_helper = df_helper.assign(whichvisit=['recontactvisit']*df_rows)
    # in df, drop all "recontact" columns
    # in df_helper, drop all initial visit columns, then rename the recontact columns
    cols_recontact = []
    cols_original = []
    for c in df.columns:
        if "recontact" in c:
            cols_recontact.append(c)
            cols_original.append(c[:-10])  
    df = df.drop(columns=cols_recontact)
    df_helper = df_helper.drop(columns=cols_original)
    cols_dict = {}
    for cname in cols_recontact:
        cols_dict[cname] = cname[:-10]
    # rename cols in df_helper
    df_helper.rename(columns=cols_dict, inplace=True)    
    mydata_withrecontact = pd.concat([df, df_helper])
    return(mydata_withrecontact)

def print_summary(obj):
    return rprint(summary(obj))

def print_short_summary(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r("print(myoutput[stringid-1])")
    ro.r("print(myoutput[stringid])")
    return (0)

def print_problematic_items(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    print('Problematic items:')
    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutput)")
    ro.r("print(myoutput[stringidsig])")   
    return (0)
    
def extract_p(invariance_output):
    ro.globalenv['invariance_output'] = invariance_output
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r('myrline <- myoutput[stringid]')
    mypline = ro.r('myrline')[0]
    my_p = mypline.split()[-1]
    return(my_p)

def check_secondary_criteria(fit_config):
    Criteria_passed = False
    ro.r("myoutputconfig <- capture.output(summary(fit_config, fit.measures=TRUE))")
    ro.r("stringid_cfi <- grep(\"Robust.*CFI\", myoutputconfig)")
    ro.r("cfi_line <- myoutputconfig[stringid_cfi]")
    cfiline = ro.r('cfi_line')[0]    
    my_cfi = cfiline.split()[-1] 
    my_cfi = float(my_cfi)
    ro.r("stringid_tli <- grep(\"Robust.*TLI\", myoutputconfig)")
    ro.r("tli_line <- myoutputconfig[stringid_tli]")
    tliline = ro.r('tli_line')[0]    
    my_tli = tliline.split()[-1] 
    my_tli = float(my_tli)
    ro.r("stringid_rmsea <- grep(\"Robust.*RMSEA\", myoutputconfig)")
    ro.r("rmsea_line <- myoutputconfig[stringid_rmsea]")
    myrmsealine = ro.r('rmsea_line')[0]
    my_rmsea = myrmsealine.split('\n')[0].split()[-1]
    my_rmsea = float(my_rmsea)   
    if (my_cfi > 0.95) and (my_tli > 0.95) and (my_rmsea < 0.06):
        Criteria_passed = True
    return (Criteria_passed, my_cfi, my_tli, my_rmsea)

def do_cfa_genpopvsenriched_invariant(mydata, whichcolumn):
    # This function runs cfa specifically between genpop and enriched, using invariant subsets that ensure invariance betwen genpop/enriched and validation
    # basically here we are testing if our invariant subsets will be invariant between enriched and genpop
    assert(whichcolumn in ['genpop_enriched'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # edit the measures for INVARIANT SUBSETS   
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop93 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop50 + hitop69 + hitop81 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop17 + hitop117 + hitop124 + hitop129 + hitop204 + hitop258',  
     'well_being': 'well_being =~hitop9 + hitop23 + hitop106 + hitop149 + hitop200 + hitop244 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata'    
    ro.globalenv['group'] = group
    # CFA
    df_data = []    
    for cln_ss, mdl in mdl_strs_new.items():
        do_metric = False
        do_scalar = False
        do_strict = False
        print("\n\n\n================================")
        print("================================")
        print(cln_ss.upper())
        print(mdl)
        print("================================")
        print("================================")
        # ====== CONFIG =====        
        ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
        fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
        ro.globalenv['fit_config'] = fit_config        
        print("----------------")
        out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                parallelType="multicore", ncpus=4)
        config_p = extract_p(out_config)
        if float(config_p) >= 0.05:
            print('CONFIG INVARIANT chisq p = ' + str(config_p))
            do_metric = True
        else:
            (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
            if passed_flag:
                print('CONFIG chisq p = ' + str(config_p))
                print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                do_metric = True
            else:
                print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
       
        if do_metric:
            # ====== METRIC =====
            fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
            ro.globalenv['fit_metric'] = fit_metric
            out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_config, 
                                                con=fit_metric, 
                                                param="loadings",
                                                parallelType="multicore", ncpus=4)
            metric_p = extract_p(out_metric)
            if float(metric_p) >= 0.05:
                print('METRIC INVARIANT chisq p = ' + str(metric_p))
                do_scalar = True
            else:
                # significant:
                print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                _ = print_problematic_items(out_metric)
            
            if do_scalar:
                # ====== SCALAR =====
                ro.globalenv['out_metric'] = out_metric
                fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                ro.globalenv['fit_scalar'] = fit_scalar
                out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_metric, 
                                                con=fit_scalar, 
                                                param=ro.StrVector(["loadings","intercepts"]),
                                                parallelType="multicore", 
                                                ncpus=4)
                scalar_p = extract_p(out_scalar)
                if float(scalar_p) >= 0.05:
                    print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    do_strict = True
                else:
                    # significant:
                    print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                    _ = print_problematic_items(out_scalar)

                if do_strict:
                    # ====== STRICT =====
                    ro.globalenv['out_scalar'] = out_scalar
                    fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                    ro.globalenv['fit_strict'] = fit_strict
                    out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_scalar, 
                                                        con=fit_strict, 
                                                        param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                    strict_p = extract_p(out_strict)
                    if float(strict_p) >= 0.05:
                        print('STRICT INVARIANT chisq p = ' + str(strict_p))
                    else:
                        # significant:
                        print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                        _ = print_problematic_items(out_strict)

                else:
                    print('STRICT N/A')
                    strict_p = 'NA'
            else:
                print('SCALAR N/A')
                print('STRICT N/A')
                scalar_p = 'NA'
                strict_p = 'NA'
        else:
            print('METRIC N/A')
            print('SCALAR N/A')
            print('STRICT N/A')
            metric_p = 'NA'
            scalar_p = 'NA'
            strict_p = 'NA'
        df_data.append({cln_ss: [config_p, metric_p, scalar_p, strict_p]})
    df = pd.DataFrame(df_data)
    print("DONE")
    return(df)
    
def do_cfa(mydata, whichcolumn):
    # this is the main function for running CFA analysis. 
    assert(whichcolumn in ['valid_enriched', 'valid_genpop', 'enriched_withrecontact', 'genpop_withrecontact', 'genpop_enriched'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',  
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    elif whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata'    
    ro.globalenv['group'] = group
    # CFA
    df_data = []    
    for cln_ss, mdl in mdl_strs_new.items():
        do_metric = False
        do_scalar = False
        do_strict = False
        print("\n\n\n================================")
        print("================================")
        print(cln_ss.upper())
        print(mdl)
        print("================================")
        print("================================")
        # ====== CONFIG =====        
        ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
        fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
        ro.globalenv['fit_config'] = fit_config        
        print("----------------")
        out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                parallelType="multicore", ncpus=4)
        config_p = extract_p(out_config)
        if float(config_p) >= 0.05:
            print('CONFIG INVARIANT chisq p = ' + str(config_p))
            do_metric = True
        else:
            (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
            if passed_flag:
                print('CONFIG chisq p = ' + str(config_p))
                print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                do_metric = True
            else:
                print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
       
        if do_metric:
            # ====== METRIC =====
            fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
            ro.globalenv['fit_metric'] = fit_metric
            out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_config, 
                                                con=fit_metric, 
                                                param="loadings",
                                                parallelType="multicore", ncpus=4)
            metric_p = extract_p(out_metric)
            if float(metric_p) >= 0.05:
                print('METRIC INVARIANT chisq p = ' + str(metric_p))
                do_scalar = True
            else:
                # significant:
                print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                _ = print_problematic_items(out_metric)
            
            if do_scalar:
                # ====== SCALAR =====
                ro.globalenv['out_metric'] = out_metric
                fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                ro.globalenv['fit_scalar'] = fit_scalar
                out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_metric, 
                                                con=fit_scalar, 
                                                param=ro.StrVector(["loadings","intercepts"]),
                                                parallelType="multicore", 
                                                ncpus=4)
                scalar_p = extract_p(out_scalar)
                if float(scalar_p) >= 0.05:
                    print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    do_strict = True
                else:
                    # significant:
                    print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                    _ = print_problematic_items(out_scalar)

                if do_strict:
                    # ====== STRICT =====
                    ro.globalenv['out_scalar'] = out_scalar
                    fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                    ro.globalenv['fit_strict'] = fit_strict
                    out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_scalar, 
                                                        con=fit_strict, 
                                                        param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                    strict_p = extract_p(out_strict)
                    if float(strict_p) >= 0.05:
                        print('STRICT INVARIANT chisq p = ' + str(strict_p))
                    else:
                        # significant:
                        print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                        _ = print_problematic_items(out_strict)

                else:
                    print('STRICT N/A')
                    strict_p = 'NA'
            else:
                print('SCALAR N/A')
                print('STRICT N/A')
                scalar_p = 'NA'
                strict_p = 'NA'
        else:
            print('METRIC N/A')
            print('SCALAR N/A')
            print('STRICT N/A')
            metric_p = 'NA'
            scalar_p = 'NA'
            strict_p = 'NA'
        df_data.append({cln_ss: [config_p, metric_p, scalar_p, strict_p]})
    df = pd.DataFrame(df_data)
    print("DONE")
    return(df)

def run_specific_cfa(mydata, whichcolumn, whichscale, whichcfa):
    # Run this function after finding an invariant subset ot items for a scale
    # (you do that with run_specific_cfa3 or run_specific_cfa3_genpopandenriched, below).
    # This function will help calculate CFA for all levels for the selected scale and subset of items in the invariant subset.
    # We need this because the functions run_specific_cfa3 and run_specific_cfa3_genpopandenriched only run cfa up to the metric level,
    # which is because these functions iterate through all subsets of items for a scale (looking for an invariant subset), which gets
    # computationally heavy. So, instead, we first find an invariant subset on the metric level and then check scalar and strict level 
    # for this subset.
    # Before running this, manually, go into the mdl_strs_new dictionary --> for the selected scale, edit the items that you want to run.
    # Then, before calling this function below, don't forget to run this cell again to load the new version of this function.
    # I recommend setting whichcfa to strict, so that it runs all levels.
    print('! DONT FORGET TO CHANGE MDL_STRS_NEW BELOW IN CODE FOR ABLATION EXPERIMENTS')
    assert(whichcolumn in ['valid_enriched', 'valid_genpop', 'enriched_withrecontact', 'genpop_withrecontact', 'genpop_enriched'])
    assert(whichscale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'hyposomnia', 'insomnia', 'panic', 'separation_insecurity',
                         'shame_guilt', 'situational_phobia', 'social_anxiety', 'well_being'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop93 + hitop182 + hitop230 + hitop246', #   hitop84 + hitop92 + hitop123 + hitop157
     'anxious_worry': 'anxious_worry =~hitop34 + hitop89 + hitop203 + hitop248', # + hitop265, 20
     'appetite_gain': 'appetite_gain =~hitop141 + hitop243 + hitop275', # + hitop120
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151', #  + hitop197
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 +  hitop114 + hitop117 + hitop124 +hitop129 + hitop204 + hitop236 + hitop258', # hitop17 + hitop22 
     'well_being': 'well_being =~hitop9 + hitop23 + hitop106 + hitop149 + hitop200 + hitop244 + hitop250 + hitop281'} # + hitop54 + hitop245
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    elif whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata' 
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        
            ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
            fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
            ro.globalenv['fit_config'] = fit_config        
            out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                    parallelType="multicore", ncpus=4)
            config_p = extract_p(out_config)
            if float(config_p) >= 0.05:
                print('CONFIG INVARIANT chisq p = ' + str(config_p))
            else:
                (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                if passed_flag:
                    print('CONFIG chisq p = ' + str(config_p))
                    print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                else:
                    print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
            if do_metric:
                # ====== METRIC =====
                fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                ro.globalenv['fit_metric'] = fit_metric
                out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_config, 
                                                    con=fit_metric, 
                                                    param="loadings",
                                                    parallelType="multicore", ncpus=4)
                metric_p = extract_p(out_metric)
                if float(metric_p) >= 0.05:
                    print('METRIC INVARIANT chisq p = ' + str(metric_p))
                else:
                    # significant:
                    print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                    ro.globalenv['out_metric'] = out_metric
                    ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                    ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringid-1])")
                    ro.r("print(myoutputmetric[stringid])")
                    # significant:
                    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringidsig])")
                if do_scalar:
                    # ====== SCALAR =====
                    ro.globalenv['out_metric'] = out_metric
                    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                    ro.globalenv['fit_scalar'] = fit_scalar
                    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_metric, 
                                                    con=fit_scalar, 
                                                    param=ro.StrVector(["loadings","intercepts"]),
                                                    parallelType="multicore", 
                                                    ncpus=4)
                    scalar_p = extract_p(out_scalar)
                    if float(scalar_p) >= 0.05:
                        print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    else:
                        # significant:
                        print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                        _ = print_problematic_items(out_scalar)
                        ro.globalenv['out_scalar'] = out_scalar
                        ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringid-1])")
                        ro.r("print(myoutputscalar[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringidsig])")
                    if do_strict:
                        # ====== STRICT =====
                        ro.globalenv['out_scalar'] = out_scalar
                        fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                        ro.globalenv['fit_strict'] = fit_strict
                        out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                            uncon=fit_scalar, 
                                                            con=fit_strict, 
                                                            param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                            parallelType="multicore", 
                                                            ncpus=4)
                        strict_p = extract_p(out_strict)
                        if float(strict_p) >= 0.05:
                            print('STRICT INVARIANT chisq p = ' + str(strict_p))
                        else:
                            # significant:
                            print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                            _ = print_problematic_items(out_strict)
                            ro.globalenv['out_strict'] = out_strict
                            ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringid-1])")
                            ro.r("print(myoutputstrict[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringidsig])")
                    else:
                        print('STRICT N/A')
                        strict_p = 'NA'
                else:
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    scalar_p = 'NA'
                    strict_p = 'NA'
            else:
                print('METRIC N/A')
                print('SCALAR N/A')
                print('STRICT N/A')
                metric_p = 'NA'
                scalar_p = 'NA'
                strict_p = 'NA'
    print("DONE")
    return(0)    
    

In [7]:
def run_specific_cfa3(mydata, whichcolumn, whichscale, whichcfa, howmanyitems):
    # This function looks for an invariant subset of items for the selected scale.
    # it iterates through all possible combinations of n items (n being specified in "howmanyitems") 
    # and outputs configural and metric cfa for every combination.
    # I recommend setting "whichcfa" to metric because ultimately we are interested in cfa up until metric, and checking
    # all four levels will double processing time.
    # Start with setting homanyitems to one less than the full amount of items in the scale (for example, set 9 for anhedonic_depression,
    # 5 for anxious_worry, etc). Then, if no invariant subset is found, set it to two less than the full amount of items, etc.
    print('THIS ONE RUNS ALL COMBINATIONS OF ITEMS')
    assert(whichcolumn in ['valid_enriched', 'valid_genpop', 'enriched_withrecontact', 'genpop_withrecontact', 'genpop_enriched'])
    assert(whichscale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'hyposomnia', 'insomnia', 'panic', 'separation_insecurity',
                         'shame_guilt', 'situational_phobia', 'social_anxiety', 'well_being'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    elif whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata' 
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        

            # loop to do ablations
            temp_items = mdl_strs_new[cln_ss]
            temp_items_items = temp_items.split("=~",1)[1]
            temp_items_items_items = temp_items_items.split(" + ")
            list_of_items = temp_items_items_items

            for com in combinations(list_of_items, howmanyitems):
                print('\nCOMBINATIONS OF ITEMS: ')
                print(com)
                mynewstring = temp_items.split("=~",1)[0] + "=~"
                for x in com:
                    mynewstring += x + ' + '
                mynewstring = mynewstring[:-3]
                print(mynewstring)
            
                ro.globalenv[cln_ss+'_mdl'] = mynewstring           
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                        #ro.globalenv['out_metric'] = out_metric
                        #ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                        #ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                        #ro.r("print(myoutputmetric[stringid-1])")
                        #ro.r("print(myoutputmetric[stringid])")
                        # significant:
                        #ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                        #ro.r("print(myoutputmetric[stringidsig])")
                    if do_scalar:
                        # ====== SCALAR =====
                        ro.globalenv['out_metric'] = out_metric
                        fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                        ro.globalenv['fit_scalar'] = fit_scalar
                        out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_metric, 
                                                        con=fit_scalar, 
                                                        param=ro.StrVector(["loadings","intercepts"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                        scalar_p = extract_p(out_scalar)
                        if float(scalar_p) >= 0.05:
                            print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                        else:
                            # significant:
                            print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                            _ = print_problematic_items(out_scalar)
                            ro.globalenv['out_scalar'] = out_scalar
                            ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                            ro.r("print(myoutputscalar[stringid-1])")
                            ro.r("print(myoutputscalar[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                            ro.r("print(myoutputscalar[stringidsig])")
                        if do_strict:
                            # ====== STRICT =====
                            ro.globalenv['out_scalar'] = out_scalar
                            fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                            ro.globalenv['fit_strict'] = fit_strict
                            out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                                uncon=fit_scalar, 
                                                                con=fit_strict, 
                                                                param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                                parallelType="multicore", 
                                                                ncpus=4)
                            strict_p = extract_p(out_strict)
                            if float(strict_p) >= 0.05:
                                print('STRICT INVARIANT chisq p = ' + str(strict_p))
                            else:
                                # significant:
                                print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                                _ = print_problematic_items(out_strict)
                                ro.globalenv['out_strict'] = out_strict
                                ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                                ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                                ro.r("print(myoutputstrict[stringid-1])")
                                ro.r("print(myoutputstrict[stringid])")
                                # significant:
                                ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                                ro.r("print(myoutputstrict[stringidsig])")
                        else:
                            print('STRICT N/A')
                            strict_p = 'NA'
                    else:
                        print('SCALAR N/A')
                        print('STRICT N/A')
                        scalar_p = 'NA'
                        strict_p = 'NA'
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'
    print("DONE")
    return(0)

In [41]:
def run_specific_cfa3_genpopandenriched(whichscale, whichcfa, howmanyitems):
    # This function looks for invariant subsets for a selected scale, checking configural and metric cfa between 
    # the validation data vs genpop AND validation vs enriched.
    # This function works ONLY UP TO THE METRIC LEVEL because we are not interested in finding subsets that are invariant 
    # on further levels in both datasets (it also gets too complicated).
    print('THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data')
    assert(whichscale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'hyposomnia', 'insomnia', 'panic', 'separation_insecurity','shame_guilt', 'situational_phobia', 'social_anxiety', 'well_being'])
    flag_found_inv_in_both = False
    
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        

            # loop to do ablations
            temp_items = mdl_strs_new[cln_ss]
            temp_items_items = temp_items.split("=~",1)[1]
            temp_items_items_items = temp_items_items.split(" + ")
            list_of_items = temp_items_items_items

            for com in combinations(list_of_items, howmanyitems):
                print('\nCOMBINATIONS OF ITEMS: ')
                print(com)
                mynewstring = temp_items.split("=~",1)[0] + "=~"
                for x in com:
                    mynewstring += x + ' + '
                mynewstring = mynewstring[:-3]
                print(mynewstring)
            
                ro.globalenv[cln_ss+'_mdl'] = mynewstring    

                # set flags for metric invariance genpop and enriched to FALSE
                metric_inv_genpop = False
                metric_inv_enriched = False
                
                # FIRST TRY GENPOP

                print('GENPOP')
                
                # to load the data in R, let's save it as csv and then load it to R
                # there may be a more elegant day to do this, but I don't know it...
                
                mydata_gp = mydata_val_genpop_norecontact
                mydata_gp.to_csv('../../data/CFA/mydata_valid_genpop.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
                group = 'whichdata'

                ro.globalenv['group'] = group
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_genpop = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'


                # NOW TRY ENRICHED

                print('ENRICHED')

                mydata_enr = mydata_val_enriched_norecontact
                mydata_enr.to_csv('../../data/CFA/mydata_valid_enriched.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
                group = 'whichdata'

                ro.globalenv['group'] = group
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))

                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_enriched = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'

                if metric_inv_genpop == True and metric_inv_enriched == True:
                    print("\n!!! THIS COMBINATION:")
                    print(com)
                    print("!!! IS INVARIANT ON BOTH GENPOP AND ENRICHED")
                    flag_found_inv_in_both = True
                        
    print("DONE")
    if flag_found_inv_in_both == True:
        print('Found at least 1 combination that is invariant in both datasets')
    else:
        print('Could not find a combination that is invariant in both datasets')
    return(0)

# Run Main Code

## Load all data

### Load item_lookup

In [9]:
item_lookup = load_item_lookup()
item_lookup
#global item_lookup

/Users/zeleninam2/miniconda3/envs/env_NEW01/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,id_cm,subscale,item,id_ht,htid
0,3.0,anhedonic depression,I had very little energy.,157,hitop157
1,9.0,separation insecurity,I worried that others would abandon me.,81,hitop81
2,16.0,anxious worry,Thoughts were racing through my head.,34,hitop34
3,19.0,well-being,It was easy for me to laugh.,54,hitop54
4,23.0,appetite gain,I stuffed myself with food.,243,hitop243
5,27.0,anhedonic depression,I was unable to enjoy things like I normally do.,182,hitop182
6,28.0,separation insecurity,I wanted other people to take care of me.,69,hitop69
7,41.0,anxious worry,I had a lot of nervous energy.,89,hitop89
8,43.0,separation insecurity,I wanted someone else to make decisions for me.,50,hitop50
9,47.0,social anxiety,I avoided performing or giving a talk in front of others.,129,hitop129


### Load Validation data

In [10]:
data_val = load_valsample()

### Load GenPop data 
### and concatinate Val + Genpop

In [11]:
data_genpop, data_genpop_norecontact = load_data('genpop', doing_checks = True, doing_remove_checks='gridinitialvisitonly')
data_genpop.shape

gridinitialvisitonly
checks_initial:
passed_checks    0.462
passed_grid      0.006
passed_list      0.458
dtype: float64
Passed checks: 269 out of 500
checks_recontact:
passed_checks_recontact    0.548
passed_grid_recontact      0.200
passed_list_recontact      0.548
dtype: float64
Passed checks: 226 out of 500
removing checks
(497, 735)
done removing checks


(497, 167)

In [12]:
data_genpop_norecontact.shape

(497, 84)

In [13]:
data_val.shape

(496, 84)

In [14]:
data_val.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


In [15]:
mydata_val_genpop = pd.concat([data_val, data_genpop])
mydata_val_genpop.shape

(993, 167)

In [16]:
mydata_val_genpop.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hitop66_recontact,hitop240_recontact,hitop222_recontact,hitop90_recontact,hitop113_recontact,hitop278_recontact,hitop203_recontact,hitop159_recontact,hitop123_recontact,hitop275_recontact,hitop268_recontact,hitop225_recontact,hitop143_recontact,hitop151_recontact,hitop181_recontact,hitop211_recontact,hitop17_recontact,hitop126_recontact,hitop5_recontact,hitop261_recontact,hitop220_recontact,hitop15_recontact,hitop72_recontact,hitop140_recontact,hitop109_recontact,hitop197_recontact,hitop104_recontact
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation,NaN,NaN,NaN,N

In [17]:
mydata_val_genpop_norecontact = pd.concat([data_val, data_genpop_norecontact])
mydata_val_genpop_norecontact.shape

(993, 84)

### Load Enriched data
### and concatinate Val + enriched

In [18]:
data_enriched, data_enriched_norecontact = load_data('enriched', doing_checks = True, doing_remove_checks='gridinitialvisitonly')
print(data_enriched.shape)

gridinitialvisitonly
checks_initial:
passed_checks    0.293548
passed_grid      0.016129
passed_list      0.290323
dtype: float64
Passed checks: 219 out of 310
checks_recontact:
passed_checks_recontact    0.412903
passed_grid_recontact      0.174194
passed_list_recontact      0.412903
dtype: float64
Passed checks: 182 out of 310
removing checks
(305, 735)
done removing checks
(305, 167)


In [19]:
mydata_val_enriched = pd.concat([data_val, data_enriched])
print(mydata_val_enriched.shape)
mydata_val_enriched_norecontact = pd.concat([data_val, data_enriched_norecontact])
print(mydata_val_enriched_norecontact.shape)

(801, 167)
(801, 84)


In [20]:
mydata_val_enriched.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hitop66_recontact,hitop240_recontact,hitop222_recontact,hitop90_recontact,hitop113_recontact,hitop278_recontact,hitop203_recontact,hitop159_recontact,hitop123_recontact,hitop275_recontact,hitop268_recontact,hitop225_recontact,hitop143_recontact,hitop151_recontact,hitop181_recontact,hitop211_recontact,hitop17_recontact,hitop126_recontact,hitop5_recontact,hitop261_recontact,hitop220_recontact,hitop15_recontact,hitop72_recontact,hitop140_recontact,hitop109_recontact,hitop197_recontact,hitop104_recontact
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation,NaN,NaN,NaN,N

In [21]:
data_genpop.head()

,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hitop66_recontact,hitop240_recontact,hitop222_recontact,hitop90_recontact,hitop113_recontact,hitop278_recontact,hitop203_recontact,hitop159_recontact,hitop123_recontact,hitop275_recontact,hitop268_recontact,hitop225_recontact,hitop143_recontact,hitop151_recontact,hitop181_recontact,hitop211_recontact,hitop17_recontact,hitop126_recontact,hitop5_recontact,hitop261_recontact,hitop220_recontact,hitop15_recontact,hitop72_recontact,hitop140_recontact,hitop109_recontact,hitop197_recontact,hitop104_recontact,whichdata
0,2,2,3,4,3,2,2,2,1,2,3,1,1,1,2,4,4,3,1,2,2,2,1,1,1,1,1,1,1,1,1,2,3,1,1,2,4,2,2,1,2,1,1,2,4,3,1,4,1,4,4,2,1,4,1,1,1,2,1,2,1,1,4,2,1,2,1,1,2,2,1,1,1,1,1,2,1,1,1,2,1,1,1,3,1,2,4,3,1,2,2,2,2,2,1,1,1,1,4,4,3,1,1,2,2,1,1,1,2,1,1,1,1,1,1,4,1,1,2,2,1,1,1,2,1,1,1,1,3,1,4,1,4,4,2,1,4,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,genpop
1,2,1,4,4,1,3,1,3,1,2,4,3,1,2,3,4,4,4,2,4,2,3,2,1,2,4,2,2,1,2,1,2,4,1,3,4,2,3,1,2,2,4,2,2,1,4,3,4,4,4,4,2,2,4,4,2,3,2,4,2,2,2,1,3,1,2,1,2,2,2,2,4,2,3,2,2,2,3,1,2,2,1,3,2,1,3,4,1,2,1,2,1,3,4,2,2,2,1,4,4,4,1,3,3,2,2,1,1,4,2,2,2,4,1,2,4,3,4,2,3,2,1,2,2,2,1,2,2,2,3,4,4,4,4,2,2,4,4,3,2,2,4,2,1,1,1,2,1,2,2,2,1,1,2,2,2,2,2,3,1,2,1,1,2,1,2,genpop
2,2,2,3,3,3,2,2,3,2,2,2,4,2,3,2,2,3,2,3,2,2,3,3,3,2,3,2,3,2,2,3,2,4,4,3,2,2,3,1,2,3,2,2,4,3,2,3,2,4,3,2,3,3,2,2,2,2,3,2,3,3,2,3,2,3,3,2,3,2,2,2,3,3,2,2,2,4,3,2,3,2,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,genpop
3,2,1,1,3,2,1,1,1,1,1,1,1,1,1,1,3,1,2,1,1,1,1,1,1,2,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,4,3,2,1,3,2,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,1,2,1,1,2,1,2,1,1,1,1,1,2,1,2,4,1,1,1,2,2,1,4,1,1,1,2,2,1,1,1,1,2,1,4,1,1,1,1,1,1,2,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,2,4,2,4,4,2,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,2,1,1,1,2,1,2,4,1,1,genpop
4,4,1,2,4,4,4,4,1,2,3,1,2,1,3,3,2,1,4,3,2,2,3,1,2,4,3,2,2,4,4,2,2,1,1,1,2,3,2,1,2,3,4,2,2,1,3,4,2,1,1,2,4,1,1,2,3,1,1,1,2,1,1,3,3,1,4,2,2,2,3,1,2,2,2,1,4,2,2,2,2,1,1,3,2,1,2,1,3,3,2,1,2,1,2,1,1,3,3,1,2,1,2,2,2,2,2,3,3,3,2,2,3,3,1,3,1,2,1,3,3,2,1,2,3,2,3,2,3,1,3,2,1,1,1,3,2,1,1,

In [22]:
data_genpop_recontactanalysis = prep_data_recontactanalysis(data_genpop)
data_enriched_recontactanalysis = prep_data_recontactanalysis(data_enriched)

In [23]:
mydata_val_genpop_norecontact.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


### Concatinate Genpop + Enriched

In [24]:
data_genpop_norecontact.head

<bound method NDFrame.head of      hitop157  hitop81  hitop34  hitop54  hitop243  hitop182  hitop69  \
0           2        2        3        4         3         2        2   
1           2        1        4        4         1         3        1   
2           2        2        3        3         3         2        2   
3           2        1        1        3         2         1        1   
4           4        1        2        4         4         4        4   
..        ...      ...      ...      ...       ...       ...      ...   
495         2        2        2        2         2         2        3   
496         1        1        1        1         1         1        1   
497         1        1        1        4         1         1        1   
498         3        1        2        2         2         2        2   
499         2        1        1        4         1         3        1   

     hitop89  hitop50  hitop129  hitop265  hitop124  hitop231  hitop93  \
0          2       

In [25]:
print(data_genpop_norecontact.shape)
print(data_enriched_norecontact.shape)

(497, 84)
(305, 84)


In [26]:
data_genpop_enriched = pd.concat([data_genpop_norecontact, data_enriched_norecontact])
print(data_genpop_enriched.shape)
data_genpop_enriched.whichdata.value_counts()

(802, 84)


whichdata
genpop      497
enriched    305
Name: count, dtype: int64

# Do main CFA analysis

In [91]:
# data that exists:
# - mydata_val_genpop
# - mydata_val_enriched
# - data_genpop_recontactanalysis
# - data_enriched_recontactanalysis

# whichcolumn == 'valid_genpop', 'valid_enriched', 'enriched_withrecontact', 'genpop_withrecontact':
my_df = do_cfa(mydata = mydata_val_enriched_norecontact, whichcolumn = 'valid_enriched') 

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.







ANHEDONIC_DEPRESSION
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246
----------------


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.959
METRIC INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'anhedonic_depression=~hitop92' may differ between Groups 'validation' and 'enriched'." 
[2] "Parameter 'anhedonic_depression=~hitop123' may differ between Groups 'validation' and 'enriched'."
[3] "Parameter 'anhedonic_depression=~hitop157' may differ between Groups 'validation' and 'enriched'."


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

CONFIG chisq p = 0.044
PASSED SECONDARY CRITERIA WITH CFI = 0.999 TLI = 0.999 RMSEA = 0.047
METRIC INVARIANT NOT PASSED, chisq p = 0
Problematic items:
character(0)


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR N/A
STRICT N/A



WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281
----------------


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0
PASSED SECONDARY CRITERIA WITH CFI = 0.993 TLI = 0.991 RMSEA = 0.043
METRIC INVARIANT NOT PASSED, chisq p = 0.041
Problematic items:
character(0)
SCALAR N/A
STRICT N/A
DONE


# --- Ablation experiments ---

In [88]:
# skip this cell
run_specific_cfa(mydata_val_enriched, 
                 whichcolumn = 'valid_enriched', 
                 whichscale = 'social_anxiety', 
                 whichcfa = 'metric')

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




! DONT FORGET TO CHANGE MDL_STRS_NEW BELOW IN CODE FOR ABLATION EXPERIMENTS
lets go


SOCIAL_ANXIETY
social_anxiety =~hitop1 +  hitop17 + hitop117 + hitop124


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.746
METRIC INVARIANT NOT PASSED, chisq p = 0.021
[1] "      AFI.Difference p.value"
[1] "chisq          6.982   0.021"
[1] "Parameter 'social_anxiety=~hitop117' may differ between Groups 'validation' and 'enriched'."
[2] "Parameter 'social_anxiety=~hitop124' may differ between Groups 'validation' and 'enriched'."
SCALAR N/A
STRICT N/A
DONE


0

In [66]:
# skip this cell
run_specific_cfa3(mydata = mydata_val_enriched_norecontact, 
                 whichcolumn = 'valid_enriched', 
                 whichscale = 'social_anxiety', 
                 whichcfa = 'metric',
                 howmanyitems = 6)

'''
'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}'''

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




     hitop39  hitop77  hitop84  hitop92  hitop93  hitop123  hitop157  \
0          4        3        3        3        4         1         3   
1          2        1        1        2        2         1         2   
2          1        1        1        1        1         1         1   
3          1        1        1        2        1         1         4   
4          2        3        4        3        2         3         3   
..       ...      ...      ...      ...      ...       ...       ...   
305        4        4        4        4        4         1         4   
306        2        2        2        2        3         1         4   
307        2        2        4        3        3         3         4   
308        2        2        2        2        2         2         2   
309        4        2        1        3        3         1         4   

     hitop182  hitop230  hitop246  hitop20  hitop34  hitop89  hitop203  \
0           2         2         2        3        3        2 

R[write to console]: 



RRuntimeError: Error in lav_data_full(data = data, group = group, cluster = cluster,  : 
  lavaan ERROR: grouping variable ‘valid_genpop’ not found;
  variable names found in data frame are:
  X hitop39 hitop77 hitop84 hitop92 hitop93 hitop123 hitop157 hitop182 hitop230 hitop246 hitop20 hitop34 hitop89 hitop203 hitop240 hitop248 hitop265 hitop120 hitop141 hitop243 hitop275 hitop109 hitop280 hitop283 hitop67 hitop142 hitop159 hitop189 hitop5 hitop66 hitop99 hitop181 hitop231 hitop21 hitop90 hitop95 hitop160 hitop254 hitop261 hitop268 hitop15 hitop104 hitop126 hitop211 hitop215 hitop257 hitop40 hitop50 hitop69 hitop81 hitop113 hitop136 hitop151 hitop197 hitop72 hitop140 hitop143 hitop220 hitop16 hitop165 hitop225 hitop247 hitop278 hitop1 hitop17 hitop114 hitop117 hitop124 hitop129 hitop204 hitop222 hitop236 hitop258 hitop9 hitop23 hitop54 hitop106 hitop149 hitop200 hitop244 hitop245 hitop250 hitop281 whichdata


In [67]:
# skip this cell
run_specific_cfa(mydata_val_enriched, 
                 whichcolumn = 'valid_enriched', 
                 whichscale = 'social_anxiety', 
                 whichcfa = 'metric')

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


# Ablation experiments looking for invariant subsets in BOTH datasets
## VERY computatinally heavy

In [81]:
run_specific_cfa3_genpopandenriched(whichscale = 'separation_insecurity', 
                                 whichcfa = 'metric',
                                 howmanyitems = 6)

'''
'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}'''

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data
lets go


SEPARATION_INSECURITY
separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197

COMBINATIONS OF ITEMS: 
('hitop40', 'hitop50', 'hitop69', 'hitop81', 'hitop113', 'hitop136')
separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136
GENPOP


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0
PASSED SECONDARY CRITERIA WITH CFI = 0.994 TLI = 0.99 RMSEA = 0.056


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.362
ENRICHED


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.052
METRIC INVARIANT NOT PASSED, chisq p = 0.017
DONE
Found at least 1 combination that is invariant in both datasets


"\n'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',\n'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',\n'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',\n'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',\n'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',\n'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',\n'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',\n'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',\n'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',\n'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',\n'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}"

# Which items mean what

In [ ]:
'''
'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
     '''

In [191]:
#add item IDs that you want to check here in the list
items_to_check = ['120', '141', '243', '275', '149', '200', '244', '245', '250', '281']

for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(bb)

   I could not keep myself from eating.
   I thought a lot about food.
   I stuffed myself with food.
   I ate even when I was not really hungry.
   I felt good about myself.
   I found compliments very encouraging.
   I felt optimistic.
   I looked forward to things with enjoyment.
   I felt like I had a lot of interesting things to do.
   I felt that I had a lot to look forward to.


# Genpop vs enriched 
## after having loaded data with checks only at 1st visit removewd

In [36]:
my_df = do_cfa(mydata = data_genpop_enriched, whichcolumn = 'genpop_enriched') 




ANHEDONIC_DEPRESSION
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




----------------
CONFIG INVARIANT chisq p = 0.085


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'anhedonic_depression=~hitop157' may differ between Groups 'genpop' and 'enriched'."


...

SCALAR N/A
STRICT N/A



WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281
----------------


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.002
PASSED SECONDARY CRITERIA WITH CFI = 0.999 TLI = 0.999 RMSEA = 0.045


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.209
SCALAR INVARIANT NOT PASSED, chisq p = 0
Problematic items:
character(0)
STRICT N/A
DONE


## are invariant subsets that ensure metric invariance between validation and genopop/enriched also invariant between ganpop and enriched?

In [60]:
my_df_invariant = do_cfa_genpopvsenriched_invariant(mydata = data_genpop_enriched, whichcolumn = 'genpop_enriched')




ANHEDONIC_DEPRESSION
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop93 + hitop182 + hitop230 + hitop246
----------------


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.317


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.068
SCALAR INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'hitop84~1' may differ between Groups 'genpop' and 'enriched'."


...

CONFIG INVARIANT chisq p = 0.087
METRIC INVARIANT NOT PASSED, chisq p = 0.016
Problematic items:
[1] "Parameter 'social_anxiety=~hitop117' may differ between Groups 'genpop' and 'enriched'."


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR N/A
STRICT N/A



WELL_BEING
well_being =~hitop9 + hitop23 + hitop106 + hitop149 + hitop200 + hitop244 + hitop250 + hitop281
----------------


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.192
METRIC INVARIANT NOT PASSED, chisq p = 0.03
Problematic items:
[1] "Parameter 'well_being=~hitop200' may differ between Groups 'genpop' and 'enriched'."
SCALAR N/A
STRICT N/A
DONE


In [ ]:
my_df_invariant

## Ablation

In [87]:
run_specific_cfa3(mydata = data_genpop_enriched, 
                 whichcolumn = 'genpop_enriched', 
                 whichscale = 'social_anxiety', 
                 whichcfa = 'metric',
                 howmanyitems = 8)

'''
'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}'''

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




     hitop157  hitop81  hitop34  hitop54  hitop243  hitop182  hitop69  \
0           2        2        3        4         3         2        2   
1           2        1        4        4         1         3        1   
2           2        2        3        3         3         2        2   
3           2        1        1        3         2         1        1   
4           4        1        2        4         4         4        4   
..        ...      ...      ...      ...       ...       ...      ...   
305         4        3        3        2         1         4        4   
306         4        3        3        4         1         2        3   
307         4        1        3        1         1         3        3   
308         2        2        4        2         2         2        2   
309         4        2        2        1         2         2        2   

     hitop89  hitop50  hitop129  hitop265  hitop124  hitop231  hitop93  \
0          2        1         2         3        

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.052


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT NOT PASSED, chisq p = 0
SCALAR N/A
STRICT N/A

COMBINATIONS OF ITEMS: 
('hitop114', 'hitop117', 'hitop124', 'hitop129', 'hitop204', 'hitop222', 'hitop236', 'hitop258')
social_anxiety =~hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.066
METRIC INVARIANT NOT PASSED, chisq p = 0
SCALAR N/A
STRICT N/A
DONE


"\n'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',\n'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',\n'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',\n'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',\n'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',\n'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',\n'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',\n'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',\n'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',\n'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',\n'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}"

In [91]:
#add item IDs that you want to check here in the list
items_to_check = ['1', '114', '117', '124', '129', '204', '236', '258']

for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(bb)

   I felt shy around other people.
   I had difficulty making eye contact with others.
   I felt socially awkward.
   I avoided situations in which others were likely to watch me.
   I avoided performing or giving a talk in front of others.
   I felt uncomfortable being the center of attention.
   I felt self-conscious around others.
   I found it difficult to speak up in front of others.


In [94]:
run_specific_cfa(mydata = data_genpop_enriched, 
                 whichcolumn = 'genpop_enriched', 
                 whichscale = 'social_anxiety', 
                 whichcfa = 'strict')

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




! DONT FORGET TO CHANGE MDL_STRS_NEW BELOW IN CODE FOR ABLATION EXPERIMENTS
lets go


SOCIAL_ANXIETY
social_anxiety =~hitop1 +  hitop114 + hitop117 + hitop124 +hitop129 + hitop204 + hitop236 + hitop258


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.015
PASSED SECONDARY CRITERIA WITH CFI = 0.999 TLI = 0.999 RMSEA = 0.047


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.069
SCALAR INVARIANT NOT PASSED, chisq p = 0
Problematic items:
character(0)
[1] "      AFI.Difference p.value"
[1] "chisq         16.554       0"
character(0)


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




STRICT INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'social_anxiety=~hitop114' may differ between Groups 'genpop' and 'enriched'."
[2] "Parameter 'hitop114~~hitop114' may differ between Groups 'genpop' and 'enriched'."      
[1] "      AFI.Difference p.value"
[1] "chisq         62.859       0"
[1] "Parameter 'social_anxiety=~hitop114' may differ between Groups 'genpop' and 'enriched'."
[2] "Parameter 'hitop114~~hitop114' may differ between Groups 'genpop' and 'enriched'."      
DONE


0

# which items mean what - full list

In [6]:
# this is some extra work I did for the manuscript, not essential for this analysis

In [ ]:
'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}

In [14]:
# anhedonic depression

items_to_check = ['39', '77', '84', '92', '93', '123', '157', '182', '230', '246']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop39   It felt like there wasn’t anything interesting or fun to do.
   hitop77   I didn’t look forward to seeing friends or family.
   hitop84   I felt depressed.
   hitop92   It took a lot of effort to do everyday activities.
   hitop93   Nothing seemed interesting to me.
   hitop123   Nothing made me laugh.
   hitop157   I had very little energy.
   hitop182   I was unable to enjoy things like I normally do.
   hitop230   I felt emotionally numb.
   hitop246   I was a lot less talkative than usual.


In [15]:
# anxious worry

items_to_check = ['20', '34', '89', '203', '248', '265']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop20   I felt tense.
   hitop34   Thoughts were racing through my head.
   hitop89   I had a lot of nervous energy.
   hitop203   I felt very stressed.
   hitop248   I worried about almost everything.
   hitop265   I was overwhelmed by anxiety.


In [16]:
# appetite gain

items_to_check = ['120', '141', '243', '275']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop120   I could not keep myself from eating.
   hitop141   I thought a lot about food.
   hitop243   I stuffed myself with food.
   hitop275   I ate even when I was not really hungry.


In [17]:
# separation insecurity

items_to_check = ['40', '50', '69', '81', '113', '136', '151', '197']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop40   I felt insecure about important relationships in my life.
   hitop50   I wanted someone else to make decisions for me.
   hitop69   I wanted other people to take care of me.
   hitop81   I worried that others would abandon me.
   hitop113   I could not handle rejection.
   hitop136   I often felt jealous.
   hitop151   I felt that I needed the approval of others.
   hitop197   I could not stand being alone.


In [18]:
# social anxiety
# 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',

items_to_check = ['1', '17', '114', '117', '124', '129', '204', '222', '236', '258']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop1   I felt shy around other people.
   hitop17   I was uncomfortable meeting new people.
   hitop114   I had difficulty making eye contact with others.
   hitop117   I felt socially awkward.
   hitop124   I avoided situations in which others were likely to watch me.
   hitop129   I avoided performing or giving a talk in front of others.
   hitop204   I felt uncomfortable being the center of attention.
   hitop222   I was uncomfortable entering a room when others already were present.
   hitop236   I felt self-conscious around others.
   hitop258   I found it difficult to speak up in front of others.


In [19]:
# well-being
# 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}

items_to_check = ['9', '23', '54', '106', '149', '200', '244', '245', '250', '281']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop9   I felt like I was having a lot of fun.
   hitop23   I felt cheerful.
   hitop54   It was easy for me to laugh.
   hitop106   I was proud of myself.
   hitop149   I felt good about myself.
   hitop200   I found compliments very encouraging.
   hitop244   I felt optimistic.
   hitop245   I looked forward to things with enjoyment.
   hitop250   I felt like I had a lot of interesting things to do.
   hitop281   I felt that I had a lot to look forward to.


# 3-way CFA

In [36]:
'''
mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}'''

items_to_check = ['9', '23', '54', '106', '149', '200', '244', '245', '250', '281']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop9   I felt like I was having a lot of fun.
   hitop23   I felt cheerful.
   hitop54   It was easy for me to laugh.
   hitop106   I was proud of myself.
   hitop149   I felt good about myself.
   hitop200   I found compliments very encouraging.
   hitop244   I felt optimistic.
   hitop245   I looked forward to things with enjoyment.
   hitop250   I felt like I had a lot of interesting things to do.
   hitop281   I felt that I had a lot to look forward to.


## 1. Check in already existing inv subsets (validation vs genpop/enriched) are also inv btw genpop and enriched

In [37]:
def checkinvar1(mydata, whichcolumn, whichscale, whichcfa):
    # We take the subsets that were invariant between validation and genpop/enriched
    # and test them on genpop vs enriched
    assert(whichcolumn in ['genpop_enriched'])
    assert(whichscale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'separation_insecurity', 'social_anxiety', 'well_being'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # EDITED dict - invar subsets
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop93 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'separation_insecurity': 'separation_insecurity =~hitop50 + hitop69 + hitop81 + hitop136 + hitop151 + hitop197',
     'social_anxiety': 'social_anxiety =~hitop17 + hitop117 + hitop124 + hitop129 + hitop204 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop106 + hitop149 + hitop200 + hitop244 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    elif whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata' 
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        
            ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
            fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
            ro.globalenv['fit_config'] = fit_config        
            out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                    parallelType="multicore", ncpus=4)
            config_p = extract_p(out_config)
            if float(config_p) >= 0.05:
                print('CONFIG INVARIANT chisq p = ' + str(config_p))
            else:
                (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                if passed_flag:
                    print('CONFIG chisq p = ' + str(config_p))
                    print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                else:
                    print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
            if do_metric:
                # ====== METRIC =====
                fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                ro.globalenv['fit_metric'] = fit_metric
                out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_config, 
                                                    con=fit_metric, 
                                                    param="loadings",
                                                    parallelType="multicore", ncpus=4)
                metric_p = extract_p(out_metric)
                if float(metric_p) >= 0.05:
                    print('METRIC INVARIANT chisq p = ' + str(metric_p))
                else:
                    # significant:
                    print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                    ro.globalenv['out_metric'] = out_metric
                    ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                    ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringid-1])")
                    ro.r("print(myoutputmetric[stringid])")
                    # significant:
                    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringidsig])")
                if do_scalar:
                    # ====== SCALAR =====
                    ro.globalenv['out_metric'] = out_metric
                    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                    ro.globalenv['fit_scalar'] = fit_scalar
                    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_metric, 
                                                    con=fit_scalar, 
                                                    param=ro.StrVector(["loadings","intercepts"]),
                                                    parallelType="multicore", 
                                                    ncpus=4)
                    scalar_p = extract_p(out_scalar)
                    if float(scalar_p) >= 0.05:
                        print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    else:
                        # significant:
                        print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                        _ = print_problematic_items(out_scalar)
                        ro.globalenv['out_scalar'] = out_scalar
                        ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringid-1])")
                        ro.r("print(myoutputscalar[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringidsig])")
                    if do_strict:
                        # ====== STRICT =====
                        ro.globalenv['out_scalar'] = out_scalar
                        fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                        ro.globalenv['fit_strict'] = fit_strict
                        out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                            uncon=fit_scalar, 
                                                            con=fit_strict, 
                                                            param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                            parallelType="multicore", 
                                                            ncpus=4)
                        strict_p = extract_p(out_strict)
                        if float(strict_p) >= 0.05:
                            print('STRICT INVARIANT chisq p = ' + str(strict_p))
                        else:
                            # significant:
                            print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                            _ = print_problematic_items(out_strict)
                            ro.globalenv['out_strict'] = out_strict
                            ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringid-1])")
                            ro.r("print(myoutputstrict[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringidsig])")
                    else:
                        print('STRICT N/A')
                        strict_p = 'NA'
                else:
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    scalar_p = 'NA'
                    strict_p = 'NA'
            else:
                print('METRIC N/A')
                print('SCALAR N/A')
                print('STRICT N/A')
                metric_p = 'NA'
                scalar_p = 'NA'
                strict_p = 'NA'
    print("DONE")
    return(0)    
    

In [38]:
for scale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'separation_insecurity', 'social_anxiety', 'well_being']:
    print('\n'+scale)
    checkinvar1(mydata = data_genpop_enriched, 
                 whichcolumn = 'genpop_enriched', 
                 whichscale = scale, 
                 whichcfa = 'strict')


anhedonic_depression
lets go


ANHEDONIC_DEPRESSION
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop93 + hitop182 + hitop230 + hitop246


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.317


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.068
SCALAR INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'hitop84~1' may differ between Groups 'genpop' and 'enriched'."
[1] "      AFI.Difference p.value"
[1] "chisq         25.118       0"
[1] "Parameter 'hitop84~1' may differ between Groups 'genpop' and 'enriched'."


...

CONFIG INVARIANT chisq p = 0.192
METRIC INVARIANT NOT PASSED, chisq p = 0.03
[1] "      AFI.Difference p.value"
[1] "chisq         25.186    0.03"
[1] "Parameter 'well_being=~hitop200' may differ between Groups 'genpop' and 'enriched'."


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR INVARIANT NOT PASSED, chisq p = 0
Problematic items:
character(0)
[1] "      AFI.Difference p.value"
[1] "chisq         16.149       0"
character(0)


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




STRICT INVARIANT NOT PASSED, chisq p = 0.004
Problematic items:
[1] "Parameter 'well_being=~hitop200' may differ between Groups 'genpop' and 'enriched'."
[1] "      AFI.Difference p.value"
[1] "chisq         15.051   0.004"
[1] "Parameter 'well_being=~hitop200' may differ between Groups 'genpop' and 'enriched'."
DONE


## for subscales that didn't pass the above checks (anxious worry, social anxiety, well-being):
## -- find all invariant subsets btw validation and genpop/enriched
## -- test them on genpop vs enriched

### (a) Scale - anxious worry

In [39]:
# validation vs genpop/enriched
run_specific_cfa3_genpopandenriched(whichscale = 'anxious_worry', 
                                 whichcfa = 'metric',
                                 howmanyitems = 5)

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data
lets go


ANXIOUS_WORRY
anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265

COMBINATIONS OF ITEMS: 
('hitop20', 'hitop34', 'hitop89', 'hitop203', 'hitop248')
anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248
GENPOP


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.419


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT NOT PASSED, chisq p = 0.001
ENRICHED


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.824
METRIC INVARIANT chisq p = 0.135
DONE
Found at least 1 combination that is invariant in both datasets


0

In [36]:
'''
mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}'''

items_to_check = ['9', '23', '54', '106', '149', '200', '244', '245', '250', '281']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop9   I felt like I was having a lot of fun.
   hitop23   I felt cheerful.
   hitop54   It was easy for me to laugh.
   hitop106   I was proud of myself.
   hitop149   I felt good about myself.
   hitop200   I found compliments very encouraging.
   hitop244   I felt optimistic.
   hitop245   I looked forward to things with enjoyment.
   hitop250   I felt like I had a lot of interesting things to do.
   hitop281   I felt that I had a lot to look forward to.


In [ ]:
# There was just one option, and it was not inv on genpop vs enriched.
# FLAGGING FOR BRUTEFORCING

### (b) Scale - social anxiety

In [42]:
# validation vs genpop/enriched
run_specific_cfa3_genpopandenriched(whichscale = 'social_anxiety', 
                                 whichcfa = 'metric',
                                 howmanyitems = 6)

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data
lets go


SOCIAL_ANXIETY
social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258

COMBINATIONS OF ITEMS: 
('hitop1', 'hitop17', 'hitop114', 'hitop117', 'hitop124', 'hitop129')
social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129
GENPOP


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT NOT PASSED, chisq p = 0.001


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT NOT PASSED, chisq p = 0.03
ENRICHED


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.774
METRIC INVARIANT NOT PASSED, chisq p = 0.005
DONE
Found at least 1 combination that is invariant in both datasets


0

In [43]:
# check the 2-way options - if they are invariant on the third leg
# this is gonna be an ugly way to do that but I'm pressed with time

#### Social anxiety - 2-way CFA option 1

"!!! THIS COMBINATION:
('hitop1', 'hitop17', 'hitop114', 'hitop129', 'hitop204', 'hitop258')
!!! IS INVARIANT ON BOTH GENPOP AND ENRICHED"

In [44]:
def checkinvar_socanx1(mydata, whichcolumn, whichscale, whichcfa):
    # We take the subsets that were invariant between validation and genpop/enriched
    # and test them on genpop vs enriched
    assert(whichcolumn in ['genpop_enriched'])
    assert(whichscale in ['social_anxiety'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # EDITED dict - invar subsets
    mdl_strs_new = {'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop129 + hitop204 + hitop258'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata' 
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        
            ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
            fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
            ro.globalenv['fit_config'] = fit_config        
            out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                    parallelType="multicore", ncpus=4)
            config_p = extract_p(out_config)
            if float(config_p) >= 0.05:
                print('CONFIG INVARIANT chisq p = ' + str(config_p))
            else:
                (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                if passed_flag:
                    print('CONFIG chisq p = ' + str(config_p))
                    print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                else:
                    print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
            if do_metric:
                # ====== METRIC =====
                fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                ro.globalenv['fit_metric'] = fit_metric
                out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_config, 
                                                    con=fit_metric, 
                                                    param="loadings",
                                                    parallelType="multicore", ncpus=4)
                metric_p = extract_p(out_metric)
                if float(metric_p) >= 0.05:
                    print('METRIC INVARIANT chisq p = ' + str(metric_p))
                else:
                    # significant:
                    print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                    ro.globalenv['out_metric'] = out_metric
                    ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                    ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringid-1])")
                    ro.r("print(myoutputmetric[stringid])")
                    # significant:
                    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringidsig])")
                if do_scalar:
                    # ====== SCALAR =====
                    ro.globalenv['out_metric'] = out_metric
                    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                    ro.globalenv['fit_scalar'] = fit_scalar
                    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_metric, 
                                                    con=fit_scalar, 
                                                    param=ro.StrVector(["loadings","intercepts"]),
                                                    parallelType="multicore", 
                                                    ncpus=4)
                    scalar_p = extract_p(out_scalar)
                    if float(scalar_p) >= 0.05:
                        print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    else:
                        # significant:
                        print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                        _ = print_problematic_items(out_scalar)
                        ro.globalenv['out_scalar'] = out_scalar
                        ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringid-1])")
                        ro.r("print(myoutputscalar[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringidsig])")
                    if do_strict:
                        # ====== STRICT =====
                        ro.globalenv['out_scalar'] = out_scalar
                        fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                        ro.globalenv['fit_strict'] = fit_strict
                        out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                            uncon=fit_scalar, 
                                                            con=fit_strict, 
                                                            param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                            parallelType="multicore", 
                                                            ncpus=4)
                        strict_p = extract_p(out_strict)
                        if float(strict_p) >= 0.05:
                            print('STRICT INVARIANT chisq p = ' + str(strict_p))
                        else:
                            # significant:
                            print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                            _ = print_problematic_items(out_strict)
                            ro.globalenv['out_strict'] = out_strict
                            ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringid-1])")
                            ro.r("print(myoutputstrict[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringidsig])")
                    else:
                        print('STRICT N/A')
                        strict_p = 'NA'
                else:
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    scalar_p = 'NA'
                    strict_p = 'NA'
            else:
                print('METRIC N/A')
                print('SCALAR N/A')
                print('STRICT N/A')
                metric_p = 'NA'
                scalar_p = 'NA'
                strict_p = 'NA'
    print("DONE")
    return(0)    
    

In [45]:
checkinvar_socanx1(mydata = data_genpop_enriched, whichcolumn = 'genpop_enriched', whichscale = 'social_anxiety', whichcfa = 'strict')

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




lets go


SOCIAL_ANXIETY
social_anxiety =~hitop1 + hitop17 + hitop114 + hitop129 + hitop204 + hitop258


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.183


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.267
SCALAR INVARIANT NOT PASSED, chisq p = 0.005
Problematic items:
character(0)
[1] "      AFI.Difference p.value"
[1] "chisq          7.208   0.005"
character(0)


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




STRICT INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'hitop114~~hitop114' may differ between Groups 'genpop' and 'enriched'."
[1] "      AFI.Difference p.value"
[1] "chisq         38.934       0"
[1] "Parameter 'hitop114~~hitop114' may differ between Groups 'genpop' and 'enriched'."
DONE


0

In [48]:
#'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
#social_anxiety =~hitop1 + hitop17 + hitop114 + hitop129 + hitop204 + hitop258

#kept
items_to_check = ['1', '17', '114', '129', '204', '258']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)
print('\n')
#removed
items_to_check = ['117', '124', '222', '236']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop1   I felt shy around other people.
   hitop17   I was uncomfortable meeting new people.
   hitop114   I had difficulty making eye contact with others.
   hitop129   I avoided performing or giving a talk in front of others.
   hitop204   I felt uncomfortable being the center of attention.
   hitop258   I found it difficult to speak up in front of others.


   hitop117   I felt socially awkward.
   hitop124   I avoided situations in which others were likely to watch me.
   hitop222   I was uncomfortable entering a room when others already were present.
   hitop236   I felt self-conscious around others.


#### Social anxiety - 2-way CFA option 2 (out of three)

!!! THIS COMBINATION:
('hitop17', 'hitop117', 'hitop124', 'hitop129', 'hitop204', 'hitop236')
!!! IS INVARIANT ON BOTH GENPOP AND ENRICHED

In [49]:
#'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
#('hitop17', 'hitop117', 'hitop124', 'hitop129', 'hitop204', 'hitop236')

#kept
items_to_check = ['17', '117', '124', '129', '204', '236']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)
print('\n')
#removed
items_to_check = ['1', '114', '222', '258']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop17   I was uncomfortable meeting new people.
   hitop117   I felt socially awkward.
   hitop124   I avoided situations in which others were likely to watch me.
   hitop129   I avoided performing or giving a talk in front of others.
   hitop204   I felt uncomfortable being the center of attention.
   hitop236   I felt self-conscious around others.


   hitop1   I felt shy around other people.
   hitop114   I had difficulty making eye contact with others.
   hitop222   I was uncomfortable entering a room when others already were present.
   hitop258   I found it difficult to speak up in front of others.


In [50]:
def checkinvar_socanx2(mydata, whichcolumn, whichscale, whichcfa):
    # We take the subsets that were invariant between validation and genpop/enriched
    # and test them on genpop vs enriched
    assert(whichcolumn in ['genpop_enriched'])
    assert(whichscale in ['social_anxiety'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # EDITED dict - invar subsets
    mdl_strs_new = {'social_anxiety': 'social_anxiety =~hitop17 + hitop117 + hitop124 + hitop129 + hitop204 + hitop236'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata' 
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        
            ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
            fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
            ro.globalenv['fit_config'] = fit_config        
            out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                    parallelType="multicore", ncpus=4)
            config_p = extract_p(out_config)
            if float(config_p) >= 0.05:
                print('CONFIG INVARIANT chisq p = ' + str(config_p))
            else:
                (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                if passed_flag:
                    print('CONFIG chisq p = ' + str(config_p))
                    print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                else:
                    print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
            if do_metric:
                # ====== METRIC =====
                fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                ro.globalenv['fit_metric'] = fit_metric
                out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_config, 
                                                    con=fit_metric, 
                                                    param="loadings",
                                                    parallelType="multicore", ncpus=4)
                metric_p = extract_p(out_metric)
                if float(metric_p) >= 0.05:
                    print('METRIC INVARIANT chisq p = ' + str(metric_p))
                else:
                    # significant:
                    print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                    ro.globalenv['out_metric'] = out_metric
                    ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                    ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringid-1])")
                    ro.r("print(myoutputmetric[stringid])")
                    # significant:
                    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringidsig])")
                if do_scalar:
                    # ====== SCALAR =====
                    ro.globalenv['out_metric'] = out_metric
                    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                    ro.globalenv['fit_scalar'] = fit_scalar
                    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_metric, 
                                                    con=fit_scalar, 
                                                    param=ro.StrVector(["loadings","intercepts"]),
                                                    parallelType="multicore", 
                                                    ncpus=4)
                    scalar_p = extract_p(out_scalar)
                    if float(scalar_p) >= 0.05:
                        print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    else:
                        # significant:
                        print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                        _ = print_problematic_items(out_scalar)
                        ro.globalenv['out_scalar'] = out_scalar
                        ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringid-1])")
                        ro.r("print(myoutputscalar[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringidsig])")
                    if do_strict:
                        # ====== STRICT =====
                        ro.globalenv['out_scalar'] = out_scalar
                        fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                        ro.globalenv['fit_strict'] = fit_strict
                        out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                            uncon=fit_scalar, 
                                                            con=fit_strict, 
                                                            param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                            parallelType="multicore", 
                                                            ncpus=4)
                        strict_p = extract_p(out_strict)
                        if float(strict_p) >= 0.05:
                            print('STRICT INVARIANT chisq p = ' + str(strict_p))
                        else:
                            # significant:
                            print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                            _ = print_problematic_items(out_strict)
                            ro.globalenv['out_strict'] = out_strict
                            ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringid-1])")
                            ro.r("print(myoutputstrict[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringidsig])")
                    else:
                        print('STRICT N/A')
                        strict_p = 'NA'
                else:
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    scalar_p = 'NA'
                    strict_p = 'NA'
            else:
                print('METRIC N/A')
                print('SCALAR N/A')
                print('STRICT N/A')
                metric_p = 'NA'
                scalar_p = 'NA'
                strict_p = 'NA'
    print("DONE")
    return(0)    
    

In [51]:
checkinvar_socanx2(mydata = data_genpop_enriched, whichcolumn = 'genpop_enriched', whichscale = 'social_anxiety', whichcfa = 'strict')

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




lets go


SOCIAL_ANXIETY
social_anxiety =~hitop17 + hitop117 + hitop124 + hitop129 + hitop204 + hitop236


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.023
PASSED SECONDARY CRITERIA WITH CFI = 0.996 TLI = 0.993 RMSEA = 0.043
METRIC INVARIANT NOT PASSED, chisq p = 0.03
[1] "      AFI.Difference p.value"
[1] "chisq         15.335    0.03"
character(0)


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR INVARIANT NOT PASSED, chisq p = 0.006
Problematic items:
character(0)
[1] "      AFI.Difference p.value"
[1] "chisq          6.685   0.006"
character(0)


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




STRICT INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'hitop17~~hitop17' may differ between Groups 'genpop' and 'enriched'."  
[2] "Parameter 'hitop129~~hitop129' may differ between Groups 'genpop' and 'enriched'."
[1] "      AFI.Difference p.value"
[1] "chisq         43.949       0"
[1] "Parameter 'hitop17~~hitop17' may differ between Groups 'genpop' and 'enriched'."  
[2] "Parameter 'hitop129~~hitop129' may differ between Groups 'genpop' and 'enriched'."
DONE


0

#### Social anxiety - 2-way CFA option 3 (out of 3)

('hitop114', 'hitop129', 'hitop204', 'hitop222', 'hitop236', 'hitop258')

In [52]:
#'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
# ('hitop114', 'hitop129', 'hitop204', 'hitop222', 'hitop236', 'hitop258')

#kept
items_to_check = ['114', '129', '204', '222', '236', '258']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)
print('\n')

#removed
items_to_check = ['1', '17', '117', '124']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop114   I had difficulty making eye contact with others.
   hitop129   I avoided performing or giving a talk in front of others.
   hitop204   I felt uncomfortable being the center of attention.
   hitop222   I was uncomfortable entering a room when others already were present.
   hitop236   I felt self-conscious around others.
   hitop258   I found it difficult to speak up in front of others.


   hitop1   I felt shy around other people.
   hitop17   I was uncomfortable meeting new people.
   hitop117   I felt socially awkward.
   hitop124   I avoided situations in which others were likely to watch me.


In [53]:
def checkinvar_socanx3(mydata, whichcolumn, whichscale, whichcfa):
    # We take the subsets that were invariant between validation and genpop/enriched
    # and test them on genpop vs enriched
    assert(whichcolumn in ['genpop_enriched'])
    assert(whichscale in ['social_anxiety'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # EDITED dict - invar subsets
    mdl_strs_new = {'social_anxiety': 'social_anxiety =~hitop114 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'genpop_enriched':
        mydata.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
        ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
        group = 'whichdata' 
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        
            ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
            fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
            ro.globalenv['fit_config'] = fit_config        
            out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                    parallelType="multicore", ncpus=4)
            config_p = extract_p(out_config)
            if float(config_p) >= 0.05:
                print('CONFIG INVARIANT chisq p = ' + str(config_p))
            else:
                (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                if passed_flag:
                    print('CONFIG chisq p = ' + str(config_p))
                    print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                else:
                    print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
            if do_metric:
                # ====== METRIC =====
                fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                ro.globalenv['fit_metric'] = fit_metric
                out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_config, 
                                                    con=fit_metric, 
                                                    param="loadings",
                                                    parallelType="multicore", ncpus=4)
                metric_p = extract_p(out_metric)
                if float(metric_p) >= 0.05:
                    print('METRIC INVARIANT chisq p = ' + str(metric_p))
                else:
                    # significant:
                    print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                    ro.globalenv['out_metric'] = out_metric
                    ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                    ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringid-1])")
                    ro.r("print(myoutputmetric[stringid])")
                    # significant:
                    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringidsig])")
                if do_scalar:
                    # ====== SCALAR =====
                    ro.globalenv['out_metric'] = out_metric
                    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                    ro.globalenv['fit_scalar'] = fit_scalar
                    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_metric, 
                                                    con=fit_scalar, 
                                                    param=ro.StrVector(["loadings","intercepts"]),
                                                    parallelType="multicore", 
                                                    ncpus=4)
                    scalar_p = extract_p(out_scalar)
                    if float(scalar_p) >= 0.05:
                        print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    else:
                        # significant:
                        print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                        _ = print_problematic_items(out_scalar)
                        ro.globalenv['out_scalar'] = out_scalar
                        ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringid-1])")
                        ro.r("print(myoutputscalar[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringidsig])")
                    if do_strict:
                        # ====== STRICT =====
                        ro.globalenv['out_scalar'] = out_scalar
                        fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                        ro.globalenv['fit_strict'] = fit_strict
                        out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                            uncon=fit_scalar, 
                                                            con=fit_strict, 
                                                            param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                            parallelType="multicore", 
                                                            ncpus=4)
                        strict_p = extract_p(out_strict)
                        if float(strict_p) >= 0.05:
                            print('STRICT INVARIANT chisq p = ' + str(strict_p))
                        else:
                            # significant:
                            print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                            _ = print_problematic_items(out_strict)
                            ro.globalenv['out_strict'] = out_strict
                            ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringid-1])")
                            ro.r("print(myoutputstrict[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringidsig])")
                    else:
                        print('STRICT N/A')
                        strict_p = 'NA'
                else:
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    scalar_p = 'NA'
                    strict_p = 'NA'
            else:
                print('METRIC N/A')
                print('SCALAR N/A')
                print('STRICT N/A')
                metric_p = 'NA'
                scalar_p = 'NA'
                strict_p = 'NA'
    print("DONE")
    return(0)    
    

In [54]:
checkinvar_socanx3(mydata = data_genpop_enriched, whichcolumn = 'genpop_enriched', whichscale = 'social_anxiety', whichcfa = 'strict')

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




lets go


SOCIAL_ANXIETY
social_anxiety =~hitop114 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.915
METRIC INVARIANT NOT PASSED, chisq p = 0
[1] "      AFI.Difference p.value"
[1] "chisq         23.127       0"
[1] "Parameter 'social_anxiety=~hitop222' may differ between Groups 'genpop' and 'enriched'."
[2] "Parameter 'social_anxiety=~hitop236' may differ between Groups 'genpop' and 'enriched'."


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'social_anxiety=~hitop222' may differ between Groups 'genpop' and 'enriched'."
[1] "      AFI.Difference p.value"
[1] "chisq         13.462       0"
[1] "Parameter 'social_anxiety=~hitop222' may differ between Groups 'genpop' and 'enriched'."


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




STRICT INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'social_anxiety=~hitop204' may differ between Groups 'genpop' and 'enriched'."
[2] "Parameter 'social_anxiety=~hitop222' may differ between Groups 'genpop' and 'enriched'."
[3] "Parameter 'social_anxiety=~hitop236' may differ between Groups 'genpop' and 'enriched'."
[4] "Parameter 'hitop114~~hitop114' may differ between Groups 'genpop' and 'enriched'."      
[5] "Parameter 'hitop222~~hitop222' may differ between Groups 'genpop' and 'enriched'."      
[1] "      AFI.Difference p.value"
[1] "chisq         60.824       0"
[1] "Parameter 'social_anxiety=~hitop204' may differ between Groups 'genpop' and 'enriched'."
[2] "Parameter 'social_anxiety=~hitop222' may differ between Groups 'genpop' and 'enriched'."
[3] "Parameter 'social_anxiety=~hitop236' may differ between Groups 'genpop' and 'enriched'."
[4] "Parameter 'hitop114~~hitop114' may differ between Groups 'genpop' and 'enriched'."      
[5] "Parameter 'hitop222

0

### (c) Scale - well-being

In [56]:
# validation vs genpop/enriched
run_specific_cfa3_genpopandenriched(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 8)

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data
lets go


WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281

COMBINATIONS OF ITEMS: 
('hitop9', 'hitop23', 'hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop244', 'hitop245')
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245
GENPOP


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.013
PASSED SECONDARY CRITERIA WITH CFI = 0.995 TLI = 0.994 RMSEA = 0.04


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT NOT PASSED, chisq p = 0.005
ENRICHED


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0
PASSED SECONDARY CRITERIA WITH CFI = 0.992 TLI = 0.988 RMSEA = 0.048
METRIC INVARIANT NOT PASSED, chisq p = 0.012
DONE
Found at least 1 combination that is invariant in both datasets


0

In [ ]:
#well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}

items_to_check = ['9', '23', '54', '106', '149', '200', '244', '245', '250', '281']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

In [ ]:
# there was just 1 combination, and it's not invariant. I need to bruteforce it.

# Bruteforcing 3-way CFA

function for anx worry and well being

In [69]:
def three_way_cfa_bruteforce(whichscale, whichcfa, howmanyitems):
    print('THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data')
    assert(whichscale in ['anxious_worry', 'well_being'])
    flag_found_inv_in_all_three = False
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    mdl_strs_new = {'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        

            # loop to do ablations
            temp_items = mdl_strs_new[cln_ss]
            temp_items_items = temp_items.split("=~",1)[1]
            temp_items_items_items = temp_items_items.split(" + ")
            list_of_items = temp_items_items_items

            for com in combinations(list_of_items, howmanyitems):
                print('\nCOMBINATIONS OF ITEMS: ')
                print(com)
                mynewstring = temp_items.split("=~",1)[0] + "=~"
                for x in com:
                    mynewstring += x + ' + '
                mynewstring = mynewstring[:-3]
                print(mynewstring)
            
                ro.globalenv[cln_ss+'_mdl'] = mynewstring    

                # set flags for metric invariance genpop and enriched to FALSE
                metric_inv_genpop = False
                metric_inv_enriched = False
                metric_inv_genpopvsenriched = False
                
                # FIRST TRY GENPOP

                print('VALID VS GENPOP')
                
                # to load the data in R, let's save it as csv and then load it to R
                # there may be a more elegant day to do this, but I don't know it...
                
                mydata_gp = mydata_val_genpop_norecontact
                mydata_gp.to_csv('../../data/CFA/mydata_valid_genpop.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
                group = 'whichdata'

                ro.globalenv['group'] = group
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_genpop = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'


                # NOW TRY ENRICHED

                print('VALID VS ENRICHED')

                mydata_enr = mydata_val_enriched_norecontact
                mydata_enr.to_csv('../../data/CFA/mydata_valid_enriched.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
                group = 'whichdata'

                ro.globalenv['group'] = group
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))

                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_enriched = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'

                
                # NOW TRY ENRICHED VS GENPOP

                print('GENPOP VS ENRICHED')

                mydata_gpvsen = data_genpop_enriched
                mydata_gpvsen.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
                group = 'whichdata' 
                ro.globalenv['group'] = group
                
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))

                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_enriched = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'


                if metric_inv_genpop == True and metric_inv_enriched == True and metric_inv_genpopvsenriched == True:
                    print("\n!!! THIS COMBINATION:")
                    print(com)
                    print("!!! PASSES 3-WAY INVARIANCE")
                    flag_found_inv_in_all_three = True
                        
    print("DONE")
    
    if flag_found_inv_in_all_three == True:
        print('Found at least 1 combination that is 3-way invariant')
    else:
        print('Could not find a combination that is invariant in both datasets')
    return(0)

In [67]:
## running bruteforcing on both anxious worry and well-being in one cell

In [70]:
print ('\n\n-----ANXIOUS WORRY-----\n\n')

print ('\n\nRUNNING ANXIOUS WORRY FOR 4 ITEMS (THATS N-2)\n\n')
three_way_cfa_bruteforce(whichscale = 'anxious_worry', 
                                 whichcfa = 'metric',
                                 howmanyitems = 4)

print ('\n\njust to be safe - queuing ANXIOUS WORRY FOR 3 ITEMS (THATS N-3)\n\n')
three_way_cfa_bruteforce(whichscale = 'anxious_worry', 
                                 whichcfa = 'metric',
                                 howmanyitems = 3)


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.






-----ANXIOUS WORRY-----




RUNNING ANXIOUS WORRY FOR 4 ITEMS (THATS N-2)


THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data
lets go


ANXIOUS_WORRY
anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265

COMBINATIONS OF ITEMS: 
('hitop20', 'hitop34', 'hitop89', 'hitop203')
anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203
VALID VS GENPOP


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.074


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

METRIC INVARIANT chisq p = 0.068

COMBINATIONS OF ITEMS: 
('hitop9', 'hitop23', 'hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop281')
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop281
VALID VS GENPOP


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.019
PASSED SECONDARY CRITERIA WITH CFI = 0.994 TLI = 0.991 RMSEA = 0.048


R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



RRuntimeError: 

In [ ]:
print ('\n\n-----WELL-BEING-----\n\n')
print ('\n\nRUNNING WELL BEING FOR 7 ITEMS (THATS N-3)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 7)

print ('\n\njust to be safe - queuing WELL BEING FOR 6 ITEMS (THATS N-4)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 6)

print ('\n\nvery worst case - queuing WELL BEING FOR 5 ITEMS (THATS N-5)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 5)

print ('\n\nvery very worst case - queuing WELL BEING FOR 4 ITEMS (THATS N-6)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 4)

print ('\n\napocalypsis - queuing WELL BEING FOR 3 ITEMS (THATS N-7)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 3)

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.






-----WELL-BEING-----




RUNNING WELL BEING FOR 7 ITEMS (THATS N-3)


THIS ONE RUNS ALL COMBINATIONS OF ITEMS and checks if a combination is invariant in both genpop AND enriched data
lets go


WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281

COMBINATIONS OF ITEMS: 
('hitop9', 'hitop23', 'hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop244')
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244
VALID VS GENPOP


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.104


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

CONFIG INVARIANT chisq p = 1


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.122
VALID VS ENRICHED


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.342
